<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/litneet64/Lab-2/litneet64_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix Factorization**
**Pablo Aravena**

# **Abstract**

 We are tasked with the implementation of the QR Decomposition, a direct solver for the equation $Ax = b$ and also for the sparse Matrix multiplication with a vector.

# **About the code**

In [0]:
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Pablo Aravena (pjan2@kth.se)

# Based on the template by Johan Hoffman (jhoffman@kth.se)
# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [0]:
# Load neccessary modules.
from google.colab import files

import numpy as np
from scipy.sparse import csr_matrix
import unittest
import random as rd
from math import isclose

# **Introduction**

In this report we implemented 3 methods, one for an efficient multiplication between sparse matrices on $CSR$ format and vectors, another one for the $QR$ Decomposition, for which we chose the $Gram-Schmidt$ process to get it done, and lastly, a direct solver for the linear equation $Ax = b$, for which we re-used the $QR$ decomposition method used previously instead of going for another decomposition like the $LU$ for this task. 

The results are then showed after running regular tests taking advantage of some already implemented numpy methods.

# **Methods**

#### Sparse Matrix-Vector Product
This method should recieve an array of the matrix in $CSR$ format, it follows the algorithm shown on the lecture notes as seen in $Algorithm$ $5.9$.

In [0]:
# method for the sparse matrix-vector product
def sparseMatrixVectorProduct(A, x):
    val, col_idx, row_ptr,  = A[0], A[1], A[2]
    
    # create result vector
    b = np.zeros(x.shape[0])
    
    # every row index
    for i in range(x.shape[0]):
        # only take non-null elements
        for j in range(row_ptr[i], row_ptr[i + 1]):
            b[i] += val[j] * x[col_idx[j]]
    
    return b



#### QR Decomposition

Following the algorithm found on the lecture notes ($Algorithm$ $5.3$), we implement the $QR$ Decomposition based on the Gram-Schmidt process with a slight modification related to making and using a copy of the $A$ matrix. 


In [0]:
# QR Factorization
def factQR(A):
    dim = A.shape[0]
    
    # create empty Q, R matrices
    R = np.zeros((dim, dim))
    Q = np.zeros((dim, dim))
    
    # we need this otherwise the A matrix would end up being modified
    A_copy = np.copy(A)
    
    # Gram-Schmidt process
    for i in range(dim):
        R[i,i] = np.linalg.norm(A_copy[:,i])
        Q[:,i] = A_copy[:,i] / R[i,i]
        
        for j in range(i+1, dim):
            R[i,j] = np.dot(Q[:,i], A[:,j])
            A_copy[:,j] -= R[i,j] * Q[:,i]
            
    return Q, R


#### Direct Solver for Ax = b
  For this method we took advantage of the QR Decomposition:
  
\begin{equation*}
    A = QR \implies A^{-1} = Q^{-1} R^{-1}
\end{equation*}

And $Q^{-1} = Q^{T}$ as $Q$ is an orthonormal matrix. Then, to get the $R^{-1}$ matrix we can just invert $R$ using backwards substitution, as it is an upper-triangular matrix. Following the algorithm found on the lecture notes ($Algorithm$ $5.1$) and using the next formula:

\begin{equation*}
    x_i = \left( b_i - \sum_{j = i+1}^n a_{ij}x_j \right) \cdot \frac{1}{a_{ii}}
\end{equation*}

We use backwards substitution on:

\begin{equation*}
    Ax = b \implies QRx = b \implies Rx = Q^{-1}b
\end{equation*}

In [0]:
# solve the linear system Ax = b using the QR decomposition
def directSolver(A, b):
    # get the number of rows for A and the Q and R matrices
    dim = A.shape[0]
    Q, R = factQR(A)
    
    # create sol vector
    x = np.zeros((dim, 1))
    
    # get the inverse of Q transposing it and the new b vector
    Q_inv = np.transpose(Q)
    new_b = Q_inv @ b
    
    # backwards substitution to get x
    x[dim - 1, 0] = new_b[dim - 1] / R[dim-1, dim-1]
    
    for i in reversed(range(dim-1)):
        curr_sum = 0
        for j in range(i+1, dim):
            curr_sum += R[i,j] * x[j, 0]
            
        x[i, 0] = (new_b[i] - curr_sum) / R[i,i]
        
    return x
    



#### Tester class

We implemented a class inheriting from the `unittest` framework as to run some test cases. These are bulk tests with randomized dimensions and elements for the matrices. The chosen number of test cases is $1.000$. 

For the Sparse Matrix-Vector multiplication, random square matrices were created, which where later transformed into sparse ones at random places (making some elements $0$) and also random vectors were made. Later, our multiplication result was tested against the numpy multiplication used for dense matrices.

To test the $QR$ Decomposition, the residuals for $||QR - A||$ and $||Q^{-1}Q - I||$ were compared to $0$ (with an error tolerance within $10^{-10}$). A similar approach was taken with the direct solver residual, where $||Ax - b||$ was compared then to $0$, after getting our own result for $x$.


In [0]:
# matrix tester class
class MatrixTester(unittest.TestCase):
    # number of test cases and maximum number of rows/cols
    TEST_CASES = 1000
    MAX_ROWS = 10
    
    # sparse matrix multiplication tester
    def testSparseMult(self):
        # go through 1.000 test cases
        for i in range(self.TEST_CASES):
            # get random number of rows and cols
            dims = rd.randint(2, self.MAX_ROWS)
            
            # generate random square matrix and vector
            A = np.random.rand(dims, dims)*2
            x = np.random.rand(dims, 1)[:,0]
            
            # and then make A a sparse matrix
            for i in range(rd.randint(0, dims)):
                for j in range(rd.randint(0, dims)):
                    A[i, j] = 0
            
            # parse it from dense form to csr format
            sparse_A = csr_matrix(A)
            sparse_A = [sparse_A.data, sparse_A.indices, sparse_A.indptr]

            our_b = sparseMatrixVectorProduct(sparse_A, x)
            real_b = A @ x
            
            # test our b vector with corresponding real value given by numpy 
            # dense-matrix multiplication (error tolerance of 1e-05)
            self.assertTrue(np.allclose(our_b, real_b))
            
    
    # random bulk tester for the jacobi iteration method
    def testQRDecomposition(self):
        # 1.000 test cases
        for i in range(self.TEST_CASES):
            # random number of rows and cols
            n = rd.randint(2, self.MAX_ROWS)
            
            # get random square matrix
            A_matr = np.random.rand(n, n)*10

            # get our Q and R matrices
            Q,R = factQR(A_matr)
            
            # get the norm for (QR - A)
            first_norm_test = np.linalg.norm(Q @ R - A_matr)

            # get the inverse of Q (same as the transpose) and the Identity matrix
            Q_inv = np.transpose(Q)
            I = np.identity(n)
            
            # and get the norm of (Q^-1 Q - I)
            sec_norm_test = np.linalg.norm(Q_inv @ Q - I)
            
            # test that ||QR - A|| = 0
            self.assertTrue(isclose(first_norm_test, 0, abs_tol = 1e-10))
            
            # test that ||Q^-1 Q - I|| = 0
            self.assertTrue(isclose(sec_norm_test, 0, abs_tol = 1e-10))
            
            
    # tester for our direct solver for Ax = b
    def testDirectSolver(self):
        # 1.000 test cases
        for i in range(self.TEST_CASES):
            # random dimension for matrix A
            n = rd.randint(2, self.MAX_ROWS)
            
            # get random square matrix and vector
            A = np.random.rand(n, n)*10
            b = np.random.rand(n, 1)*2
            
            # get our solution vector
            x = directSolver(A, b)
            
            # get the norm of (Ax - b)
            residual = np.linalg.norm(A @ x - b)
            
            # test that the residual is close to 0 ||Ax - b|| = 0
            self.assertTrue(isclose(residual, 0, abs_tol = 1e-10))
        
        

# **Results**

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

# **Discussion**

The results shows that no errors were caused by the tests, proving they work as intended. As we were given multiple choices to implement the QR Decomposition and the Direct Solver this task proved to be an interesting one.